---
---


<center><h1> Spark ML Assingment </h1></center>

----

---

#### `IMPORTING THE REQUIRED LIBRARIES`

---

In [1]:
# !pip3 install matplotlib --user
# !pip3 install seaborn --user

In [2]:
# importing the required libraries
from pyspark.sql import SparkSession
import pyspark.sql.types as tp
from pyspark.sql import functions as F

# # libraries to make plots
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [4]:
# create spark session
spark = SparkSession.builder.getOrCreate()

In [5]:
# reading the train data
train_data = spark.read.csv("ml_assignment/train.csv",inferSchema=True, header=True)

# reading the test data
test_data  = spark.read.csv("ml_assignment/test.csv", inferSchema=True, header=True)

---

# `DATA EXPLORATION`

Now, that we have read the data, the first step is to explore the data. We will try to find out the following things from the data.

---
**`1. Variable Identification`**

- Data Type of the columns

**`2. Univariate Analysis`**
- 1. Average Purchase amount?
- 2. Counting and Removing null values
- 3. How many distinct values per column?
- 4. Count category values within each of the following column:
     - Gender
     - Age
     - City_Category
     - Stay_In_Current_City_Years
     - Marital_Status

**`3. Bivariate Analysis`**

- Calculate average Purchase for each of the following columns:
     - Gender
     - Age
     - City_Category
     - Stay_In_Current_City_Years
     - Marital_Status

---

##  `VARIABLE INDENTIFICATION`

#### `Data Type of the Columns`

---

In [6]:
# data type of the columns
train_data.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [7]:
train_data.show()

+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|  Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+-----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F| 0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F| 0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|     F| 0-17|        10|            A|                         2|             0|                12|              null|              null|    1422

## `UNIVARIATE ANALYSIS`

---

#### `Exploring the Target Variable`

---

The `Target Variable` for our use-case is `Purchase` and this column has `numeric` data type.


---
`1. Average Purchase Amount`
---

In [8]:
train_data.agg(F.avg("Purchase").alias('Avg_Purchase'))

Avg_Purchase
9263.968712959126


---
` 2. Counting and Removing null values `
---

In [9]:
### null values in each column
for c in train_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    missing_values = train_data.filter(missing_values).count()
    
    # print the result
    print(c, missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 173638
Product_Category_3 383247
Purchase 0


---

So, we have not much null values in most of the columns:

 * **`User_ID`**:   0
 * **`Product_ID`**:   0
 * **`Gender`**:   0
 * **`Age`**:  0
 * **`Occupation`**:  0
 * **`City_Category`**:  0
 * **`Stay_In_Current_City_Years`**:  0
 * **`Marital_Status`**:  0
 * **`Product_Category_1`**:  0
 * **`Product_Category_2`**: 173638
 * **`Product_Category_3`**:  383247
 * **`Purchase`**:  0


---

In [10]:
product_category_1=train_data.groupBy("Product_Category_2").agg(F.count("Product_Category_2"))
product_category_1.orderBy('Product_Category_2',ascending=True).show()

+------------------+-------------------------+
|Product_Category_2|count(Product_Category_2)|
+------------------+-------------------------+
|              null|                        0|
|                 2|                    49217|
|                 3|                     2884|
|                 4|                    25677|
|                 5|                    26235|
|                 6|                    16466|
|                 7|                      626|
|                 8|                    64088|
|                 9|                     5693|
|                10|                     3043|
|                11|                    14134|
|                12|                     5528|
|                13|                    10531|
|                14|                    55108|
|                15|                    37855|
|                16|                    43255|
|                17|                    13320|
|                18|                     2770|
+------------

#### The most common values in Product_Category_2 is 8. So,we can fill the null values with null values

In [11]:
product_category_3=train_data.groupBy("Product_Category_3").agg(F.count("Product_Category_3"))
product_category_3.orderBy('Product_Category_3',ascending=True).show()

+------------------+-------------------------+
|Product_Category_3|count(Product_Category_3)|
+------------------+-------------------------+
|              null|                        0|
|                 3|                      613|
|                 4|                     1875|
|                 5|                    16658|
|                 6|                     4890|
|                 8|                    12562|
|                 9|                    11579|
|                10|                     1726|
|                11|                     1805|
|                12|                     9246|
|                13|                     5459|
|                14|                    18428|
|                15|                    28013|
|                16|                    32636|
|                17|                    16702|
|                18|                     4629|
+------------------+-------------------------+



#### The most common values in Product_Category_3 is __16__. So, we can fill the null values with __16__.

In [12]:
# fill null values in the train data
train_data = train_data.fillna({
            "Product_Category_2" : 8,
            "Product_Category_3": 16,
        })

### Check Again, and see the null values are filled or not.

In [13]:
### null values in each column
for c in train_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    missing_values = train_data.filter(missing_values).count()
    
    # print the result
    print(c, missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 0
Product_Category_3 0
Purchase 0


- Here, we can see that there are no null values are present in the dataset.
- So, we can also fill the null values in test dataset.

In [14]:
### null values in each column of test dataset.
for c in test_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    missing_values = test_data.filter(missing_values).count()
    
    # print the result
    print(c, missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 72344
Product_Category_3 162562


In [15]:
# fill null values in the train data
test_data = test_data.fillna({
            "Product_Category_2" : 8,
            "Product_Category_3": 16,
        })

In [16]:
### null values in each column of test dataset.
for c in test_data.columns:
    # define the condition
    missing_values = F.isnull(c)
    
    # filter the data with condition and count the number of data points
    missing_values = test_data.filter(missing_values).count()
    
    # print the result
    print(c, missing_values)

User_ID 0
Product_ID 0
Gender 0
Age 0
Occupation 0
City_Category 0
Stay_In_Current_City_Years 0
Marital_Status 0
Product_Category_1 0
Product_Category_2 0
Product_Category_3 0


---
` 3. How many distinct values per column? `
---

In [17]:
# distinct values in each column
train_data.agg(*(F.countDistinct(F.col(c)).alias(c) for c in train_data.columns))

User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
5891,3631,2,7,21,3,5,2,20,17,15,18105


---
` 4. Count category values within each of the following column: `
--- 
- **Gender**


In [18]:
# number of data points in each category and their percentage
train_data.groupBy("Gender").agg(F.count("Gender").alias("Count"),
                                           F.round((F.count("Gender")/train_data.count())*100, 4)
                                 .alias("Percentage")).show()



+------+------+----------+
|Gender| Count|Percentage|
+------+------+----------+
|     F|135809|   24.6895|
|     M|414259|   75.3105|
+------+------+----------+



- Here, we can see that Gender __Male__ is Higher than __Female.__

- **Age**

In [19]:
# number of data points in each category and their percentage
train_data.groupBy("Age").agg(F.count("Age").alias("Count"),
                                           F.round((F.count("Age")/train_data.count())*100, 4)
                                 .alias("Percentage")).show()

+-----+------+----------+
|  Age| Count|Percentage|
+-----+------+----------+
|18-25| 99660|   18.1178|
|26-35|219587|     39.92|
| 0-17| 15102|    2.7455|
|46-50| 45701|    8.3082|
|51-55| 38501|    6.9993|
|36-45|110013|   19.9999|
|  55+| 21504|    3.9093|
+-----+------+----------+



- Here, we can see that people age between 25 to 35 are higher number than other.
- And, Next one is age between 36 to 45 is second higher.

- **City_Category**

In [20]:
# number of data points in each category and their percentage
train_data.groupBy("City_Category").agg(F.count("City_Category").alias("Count"),
                                           F.round((F.count("City_Category")/train_data.count())*100, 4)
                                 .alias("Percentage")).show()

+-------------+------+----------+
|City_Category| Count|Percentage|
+-------------+------+----------+
|            B|231173|   42.0263|
|            C|171175|   31.1189|
|            A|147720|   26.8549|
+-------------+------+----------+



- Here, we can see that the City_Category B is higher number than others.
- And, next one is City_Category C which is second higher.

- **Stay_In_Current_City_Years**

In [21]:
# number of data points in each category and their percentage
train_data.groupBy("Stay_In_Current_City_Years").agg(F.count("Stay_In_Current_City_Years").alias("Count"),
                                           F.round((F.count("Stay_In_Current_City_Years")/train_data.count())*100, 4)
                                 .alias("Percentage")).show()

+--------------------------+------+----------+
|Stay_In_Current_City_Years| Count|Percentage|
+--------------------------+------+----------+
|                         3| 95285|   17.3224|
|                         0| 74398|   13.5252|
|                        4+| 84726|   15.4028|
|                         1|193821|   35.2358|
|                         2|101838|   18.5137|
+--------------------------+------+----------+



- Here, we can see that the most of the people stay in current city is __2__
- And, Next one the second most of the people stay in current city is __3__

- **Marital_Status**

In [22]:
train_data.groupBy("Marital_Status").agg(F.count("Marital_Status").alias("Count"),
                                           F.round((F.count("Marital_Status")/train_data.count())*100, 4)
                                 .alias("Percentage")).show()

+--------------+------+----------+
|Marital_Status| Count|Percentage|
+--------------+------+----------+
|             1|225337|   40.9653|
|             0|324731|   59.0347|
+--------------+------+----------+



- Here, we can see that the most of the people are Unmaried

---

## `BIVARIATE ANALYSIS`

---


---
` Calculate average Purchase for each of the following columns: `
--- 


- **Gender**

In [23]:
# number of datapoints with each country
train_data.groupBy("Gender").agg(F.count("Gender").alias("Gender_Count"),
                                    F.sum("Purchase").alias("Purchase by Gender")).show()

+------+------------+------------------+
|Gender|Gender_Count|Purchase by Gender|
+------+------------+------------------+
|     F|      135809|        1186232642|
|     M|      414259|        3909580100|
+------+------------+------------------+



- **Age**

In [24]:
# number of datapoints with each country
train_data.groupBy("Age").agg(F.count("Age").alias("Age_Count"),
                                    F.sum("Purchase").alias("Purchase by Age")).show()

+-----+---------+---------------+
|  Age|Age_Count|Purchase by Age|
+-----+---------+---------------+
|18-25|    99660|      913848675|
|26-35|   219587|     2031770578|
| 0-17|    15102|      134913183|
|46-50|    45701|      420843403|
|51-55|    38501|      367099644|
|36-45|   110013|     1026569884|
|  55+|    21504|      200767375|
+-----+---------+---------------+



- **City_Category**

In [25]:
train_data.groupBy("City_Category").agg(F.count("City_Category").alias("City_Category_Count"),
                                       F.sum("Purchase").alias("Purchase by City_Category")).show()

+-------------+-------------------+-------------------------+
|City_Category|City_Category_Count|Purchase by City_Category|
+-------------+-------------------+-------------------------+
|            B|             231173|               2115533605|
|            C|             171175|               1663807476|
|            A|             147720|               1316471661|
+-------------+-------------------+-------------------------+



- **Stay_In_Current_City_Years**

In [26]:
train_data.groupBy("Stay_In_Current_City_Years").agg(F.count("Stay_In_Current_City_Years").alias("Stay_In_Current_City_Years_Count"),
                                                    F.sum('Purchase').alias("Purchase by Stay_In_Current_City_Years")).show()

+--------------------------+--------------------------------+--------------------------------------+
|Stay_In_Current_City_Years|Stay_In_Current_City_Years_Count|Purchase by Stay_In_Current_City_Years|
+--------------------------+--------------------------------+--------------------------------------+
|                         3|                           95285|                             884902659|
|                         0|                           74398|                             682979229|
|                        4+|                           84726|                             785884390|
|                         1|                          193821|                            1792872533|
|                         2|                          101838|                             949173931|
+--------------------------+--------------------------------+--------------------------------------+



- **Marital_Status**

In [27]:
train_data.groupBy("Marital_Status").agg(F.count("Marital_Status").alias("Marital_Status"),
                                        F.sum("Purchase").alias("Purchase by Marital_Status")).show()

+--------------+--------------+--------------------------+
|Marital_Status|Marital_Status|Purchase by Marital_Status|
+--------------+--------------+--------------------------+
|             1|        225337|                2086885295|
|             0|        324731|                3008927447|
+--------------+--------------+--------------------------+



---

## `Pre-Processing`

---

- We will do label-encoding the categorical variable.
- We will do one-hot encoding the categorical variables.


---

---
#### `Product_ID`
##### Here, Product_ID is the ID of the different product, and hence it's a categorical variable.
---

In [28]:
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_ID')
labeller = plan_indexer.fit(train_data)


In [29]:
train_data = labeller.transform(train_data)
test_data = labeller.transform(test_data)

---

## `Label Encoding`

---

---

#### `Encoding Categorical Variables`

Categorical values need to be encoded or converted to numeric form before inputting it to an ML model.

Two ways of encoding categorical values:

**Label Encoding**

   > Assigning a unique integer to each of the categorical values
   > Done using StringIndexer

**One Hot Encoding**

   > New columns created for each of the unique value in categorical column. Values assigned 0 or 1 based on the presence of the data.
   > Requires features to be label encoded first.
   > Done using OneHotEncoderEstimator
    


----

We will do the following encoding of the categorical variables in this notebook.

 - Gender (One Hot Encode)
 - Age (One Hot Encode) 
 - City_Category (One Hot Encode)
 - Stay_in_current_city_years      (One Hot Encode)    
 
    
    
---

#### `Importing the Required Libraries`

---

In [30]:
# importing some more libraries
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator

---

#### `Creating the StringIndexer Objects`

---

In [31]:
# label encode Gender
SI_Gender = StringIndexer(inputCol= "Gender", outputCol= "gender_le" , handleInvalid="skip")

# label encode Age
SI_Age = StringIndexer(inputCol= "Age", outputCol= "age_le", handleInvalid= "skip")

# label encode City_Category
SI_City_Category      = StringIndexer(inputCol= "City_Category",      outputCol= "city_category_le", handleInvalid= "skip")

# label encode Stay_in_current_city_years
SI_Stay_in_Current_City_Years = StringIndexer(inputCol= "Stay_In_Current_City_Years", outputCol= "stay_in_current_city_years_le", handleInvalid="skip")

---

#### `Fit the StringIndexer Objects`

---

In [32]:
# label encode object Gender
SI_Gender_Obj = SI_Gender.fit(train_data)

# label encode object Age
SI_Age_Obj = SI_Age.fit(train_data)

# label encode object City_Category
SI_City_Category_Obj = SI_City_Category.fit(train_data)

# label encode object Stay_in_current_city_years
SI_Stay_in_Current_City_Years_Obj = SI_Stay_in_Current_City_Years.fit(train_data)

----

#### `Make the Transformation on the test data`

---

In [33]:
# label encode Gender
train_data_encoded = SI_Gender_Obj.transform(train_data)

# label encode Age
train_data_encoded = SI_Age_Obj.transform(train_data_encoded)

# label encode City_category
train_data_encoded = SI_City_Category_Obj.transform(train_data_encoded)

# label encode Stay_in_current_city_years
train_data_encoded = SI_Stay_in_Current_City_Years_Obj.transform(train_data_encoded)

----

#### `Make the Transformation on the test data`

---

In [34]:
test_data_encoded = SI_Gender_Obj.transform(test_data)

test_data_encoded = SI_Age_Obj.transform(test_data_encoded)

test_data_encoded = SI_City_Category_Obj.transform(test_data_encoded)

test_data_encoded = SI_Stay_in_Current_City_Years_Obj.transform(test_data_encoded)

In [35]:
# let's check the columns in the data
train_data_encoded.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'product_ID',
 'gender_le',
 'age_le',
 'city_category_le',
 'stay_in_current_city_years_le']



### `ONE-HOT ENCODING`



---

Next, we will create the object of the `OneHotEncoderEstimator` and pass the input and output columns list.


In [36]:
OHE_train = OneHotEncoderEstimator(inputCols=["gender_le",
                                              "age_le",
                                              "city_category_le",
                                              "stay_in_current_city_years_le"],
                                  outputCols=["gender_ohe",
                                              "age_ohe",
                                              "city_category_ohe",
                                              "stay_in_current_city_years_ohe"])

---

Fit & Transform the data.

---

In [37]:
# OHE object
OHE_Obj = OHE_train.fit(train_data_encoded)

# Transform train data
train_data_encoded = OHE_Obj.transform(train_data_encoded)

In [38]:
# view the one hot encoded data
train_data_encoded.select("User_ID",
                          "gender_ohe", 
                          "age_ohe",
                          "city_category_ohe",
                          "stay_in_current_city_years_ohe").show()

+-------+-------------+-------------+-----------------+------------------------------+
|User_ID|   gender_ohe|      age_ohe|city_category_ohe|stay_in_current_city_years_ohe|
+-------+-------------+-------------+-----------------+------------------------------+
|1000001|    (1,[],[])|    (6,[],[])|        (2,[],[])|                 (4,[1],[1.0])|
|1000001|    (1,[],[])|    (6,[],[])|        (2,[],[])|                 (4,[1],[1.0])|
|1000001|    (1,[],[])|    (6,[],[])|        (2,[],[])|                 (4,[1],[1.0])|
|1000001|    (1,[],[])|    (6,[],[])|        (2,[],[])|                 (4,[1],[1.0])|
|1000002|(1,[0],[1.0])|(6,[5],[1.0])|    (2,[1],[1.0])|                 (4,[3],[1.0])|
|1000003|(1,[0],[1.0])|(6,[0],[1.0])|        (2,[],[])|                 (4,[2],[1.0])|
|1000004|(1,[0],[1.0])|(6,[3],[1.0])|    (2,[0],[1.0])|                 (4,[1],[1.0])|
|1000004|(1,[0],[1.0])|(6,[3],[1.0])|    (2,[0],[1.0])|                 (4,[1],[1.0])|
|1000004|(1,[0],[1.0])|(6,[3],[1.0])|    (2

---

#### `Make the transformation on test data`

---

In [39]:
test_data_encoded = OHE_Obj.transform(test_data_encoded)

---


## `MODEL BUILDING`

Now, we have prepared the dataset and it is ready to be trained with Machine Learning models. This is a `Regression Problem`, so we will train the data on the following alogrithms.

 * **Linear Regression**
 * **Decision Tree Regression**
 * **RandomForest Regressor**

---

#### `VECTOR ASSEMBLER`

- Before passing the data into the ML model, we need to convert the required features into a Vector. We can do this using a `VectorAssembler`.

In [40]:
train_data_encoded.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'product_ID',
 'gender_le',
 'age_le',
 'city_category_le',
 'stay_in_current_city_years_le',
 'gender_ohe',
 'age_ohe',
 'city_category_ohe',
 'stay_in_current_city_years_ohe']

---

#### `Define the VectorAssembler Object`

---

In [41]:
# import the library
from pyspark.ml.feature import VectorAssembler

# create feature vector
feature_vector = VectorAssembler(inputCols= ['Marital_Status',
                                             'gender_ohe',
                                             'age_ohe',
                                             'city_category_ohe',
                                             'stay_in_current_city_years_ohe',
                                            'Product_Category_1',
                                            'Product_Category_2',
                                            'Product_Category_3'],
                                outputCol= 'feature_vector')


---

`TRANSFORM THE FEATURE VECTOR`

----

In [42]:
# transform the feature vector
train_data_encoded = feature_vector.transform(train_data_encoded)

In [43]:
# view the feature vector
train_data_encoded.select("feature_vector").show(10)

+--------------------+
|      feature_vector|
+--------------------+
|(17,[11,14,15,16]...|
|(17,[11,14,15,16]...|
|(17,[11,14,15,16]...|
|(17,[11,14,15,16]...|
|(17,[1,7,9,13,14,...|
|(17,[1,2,12,14,15...|
|(17,[0,1,5,8,11,1...|
|(17,[0,1,5,8,11,1...|
|(17,[0,1,5,8,11,1...|
|(17,[0,1,2,10,14,...|
+--------------------+
only showing top 10 rows



In [44]:
# we can see that the data type of the `feature_vector` is vector
train_data_encoded.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = false)
 |-- Product_Category_3: integer (nullable = false)
 |-- Purchase: integer (nullable = true)
 |-- product_ID: double (nullable = false)
 |-- gender_le: double (nullable = false)
 |-- age_le: double (nullable = false)
 |-- city_category_le: double (nullable = false)
 |-- stay_in_current_city_years_le: double (nullable = false)
 |-- gender_ohe: vector (nullable = true)
 |-- age_ohe: vector (nullable = true)
 |-- city_category_ohe: vector (nullable = true)
 |-- stay_in_current_city_years_ohe: vector (nullable = true)
 |-- feature_vector: vec

----

#### `Make the transformation on test data.`

----

In [45]:
# test data
test_data_encoded = feature_vector.transform(test_data_encoded)

In [46]:
train_data_encoded.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = false)
 |-- Product_Category_3: integer (nullable = false)
 |-- Purchase: integer (nullable = true)
 |-- product_ID: double (nullable = false)
 |-- gender_le: double (nullable = false)
 |-- age_le: double (nullable = false)
 |-- city_category_le: double (nullable = false)
 |-- stay_in_current_city_years_le: double (nullable = false)
 |-- gender_ohe: vector (nullable = true)
 |-- age_ohe: vector (nullable = true)
 |-- city_category_ohe: vector (nullable = true)
 |-- stay_in_current_city_years_ohe: vector (nullable = true)
 |-- feature_vector: vec

In [47]:
test_data_encoded.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = false)
 |-- Product_Category_3: integer (nullable = false)
 |-- product_ID: double (nullable = false)
 |-- gender_le: double (nullable = false)
 |-- age_le: double (nullable = false)
 |-- city_category_le: double (nullable = false)
 |-- stay_in_current_city_years_le: double (nullable = false)
 |-- gender_ohe: vector (nullable = true)
 |-- age_ohe: vector (nullable = true)
 |-- city_category_ohe: vector (nullable = true)
 |-- stay_in_current_city_years_ohe: vector (nullable = true)
 |-- feature_vector: vector (nullable = true)



---

#### `LINEAR REGRESSION`

First, we will train the model with the Linear Regression and see the `RMSE Score`.

For that let's import some more libraries.

----

In [48]:
# Import the libraries
from pyspark.ml import regression
from pyspark.ml.evaluation import RegressionEvaluator

---

First we will create the Linear Regression and pass the `feature vector` and the label column which is `Purchase`

---

In [49]:
model_LR = regression.LinearRegression(featuresCol='feature_vector', labelCol="Purchase")

---


Fit the model

---

In [50]:
model_LR = model_LR.fit(train_data_encoded)

---

Next, we will use the `RegressionEvaluator` to calculate the `rmse score` and for that we have to pass the `labelCol: Purchase` and the `metricName: rmse`

---

In [51]:
# Evaluate training data
evaluator = RegressionEvaluator(labelCol="Purchase", metricName="rmse") 
evaluator.evaluate(model_LR.transform(train_data_encoded)) 

4681.23511028296

In [52]:
# # Use a regression evaluator to evaluate the predictions
# evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse")
# evaluator.evaluate(model_RF.transform(train_data_encoded))
# # rmse = evaluator.evaluate(predictions)

In [53]:
# Evaluate test data
# evaluator.evaluate(model_LR.transform(test_data_encoded))

---

#### `DECISION TREE REGRESSION`


Next, we will train the decision tree regression model and do the same steps as we did with the linear regression.

---

In [54]:
model_DTR = regression.DecisionTreeRegressor(featuresCol= "feature_vector",  labelCol="Purchase")

In [55]:
# fit the model
model_DTR = model_DTR.fit(train_data_encoded)

In [56]:
# evaluate the model
evaluator.evaluate(model_DTR.transform(train_data_encoded))

3255.036913818846

In [57]:
# Evaluate on test data
# evaluator.evaluate(model_DTR.transform(test_data_encoded))

----

#### `RANDOM FOREST REGRESSOR`



In [58]:
model_RFR = regression.RandomForestRegressor(featuresCol= "feature_vector",  labelCol="Purchase")

In [59]:
# fit the model
model_RFR = model_RFR.fit(train_data_encoded)

In [60]:
# evaluate the model
evaluator.evaluate(model_RFR.transform(train_data_encoded))

3901.0999790553083

In [61]:
# Evaluate on test data
# evaluator.evaluate(model_RFR.transform(test_data_encoded))

---

So far, we saw the `Linear Regression`, `Decision Tree` and `RandomForestRegressor` algorithms. But we were running them using the default parameters only. So, we will tune the parameters of our model and see if it gets any better. 

First we, will do the cross validation

---

## `CROSS VALIDATION`




Let's see how to do the cross validation.

In [62]:
# import the CrossValidator and ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [63]:
# create the ParamGridBuilder
params = ParamGridBuilder().build()

#### `Linear Regression cross-validation`

In [64]:
# create the object of the Logistic Regression Model
model_LR_CV = regression.LinearRegression(featuresCol= "feature_vector",  labelCol="Purchase")

In [65]:
# create object of the cross validation model with numFolds = 3
cv = CrossValidator(estimator=model_LR_CV,
                    estimatorParamMaps=params,
                    evaluator=evaluator, # which is auc-roc score
                    numFolds=3,
                    seed=27)

In [66]:
## fit the model
cv_model = cv.fit(train_data_encoded)

In [67]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse") 
evaluator.evaluate(cv_model.transform(train_data_encoded)) 

4681.23511028296

#### `DecisionTreeRegressor cross-validation`

In [68]:
# create the object of the DecisionTreeRegressor Model
model_DTR_CV = regression.DecisionTreeRegressor(featuresCol= "feature_vector",  labelCol="Purchase")

In [69]:
# create object of the cross validation model with numFolds = 3
cv = CrossValidator(estimator=model_DTR_CV,
                    estimatorParamMaps=params,
                    evaluator=evaluator, # which is auc-roc score
                    numFolds=3,
                    seed=27)

In [70]:
## fit the model
cv_model = cv.fit(train_data_encoded)

In [71]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse") 
evaluator.evaluate(cv_model.transform(train_data_encoded)) 

3255.036913818846

#### `RandomForestRegressor cross validation`

In [72]:
# create the object of the DecisionTreeRegressor Model
model_RFR_CV = regression.RandomForestRegressor(featuresCol= "feature_vector",  labelCol="Purchase")

In [73]:
# create object of the cross validation model with numFolds = 3
cv = CrossValidator(estimator=model_RFR_CV,
                    estimatorParamMaps=params,
                    evaluator=evaluator, # which is auc-roc score
                    numFolds=3,
                    seed=27)

In [74]:
## fit the model
cv_model = cv.fit(train_data_encoded)

In [75]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse") 
evaluator.evaluate(cv_model.transform(train_data_encoded)) 

3901.0999790553083

---


## `GRID SEARCH`

Next, we will try the Grid Search where it will apply different combinations of parameter to give you the best model.

---

#### `Hyper-parameter Tunning of the model using Grid Search in Linear Regression`

In [76]:
# create parameter builder

updated_params = ParamGridBuilder() \
                .addGrid(model_LR_CV.regParam, [0.01, 0.005, 0.0001]) \
                .addGrid(model_LR_CV.elasticNetParam, [0.1, 0.001]) \
                .build()

In [77]:
# create object of the Cross Calidator with 3 folds
cv = CrossValidator(estimator=model_LR_CV,
                    estimatorParamMaps=updated_params,
                    evaluator=evaluator,
                    numFolds=3,
                    seed=27)

In [78]:
# fit the model
grid_model = cv.fit(train_data_encoded)

In [79]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse") 
evaluator.evaluate(grid_model.transform(train_data_encoded))

4681.235110344878

#### `Hyper-parameter Tunning of the model using Grid Search in DecisionTree Regression`

In [80]:
# create parameter builder

updated_params = ParamGridBuilder() \
                .addGrid(model_DTR_CV.maxDepth, [3, 5, 10, 15]) \
                .addGrid(model_DTR_CV.maxBins, [10, 20, 30, 50, 100]) \
                .build()

In [81]:
# create object of the Cross Validator with 3 folds
cv = CrossValidator(estimator=model_DTR_CV,
                    estimatorParamMaps=updated_params,
                    evaluator=evaluator,
                    numFolds=3,
                    seed=27)

In [82]:
# fit the model
grid_model = cv.fit(train_data_encoded)

In [83]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse") 
evaluator.evaluate(grid_model.transform(train_data_encoded))

2938.191126866426

#### `Hyper-parameter Tunning of the model using Grid Search in RandomForest Regression`

In [84]:

updated_params = ParamGridBuilder() \
                .addGrid(model_RFR_CV.maxDepth, [5, 10, 15]) \
                .addGrid(model_RFR_CV.numTrees, [10, 20, 30, 40]) \
                .build()

In [85]:
# create object of the Cross Validator with 3 folds
cv = CrossValidator(estimator=model_RFR_CV,
                    estimatorParamMaps=updated_params,
                    evaluator=evaluator,
                    numFolds=3,
                    seed=27)

In [86]:
# fit the model
grid_model = cv.fit(train_data_encoded)

In [87]:
# evaluate the model
evaluator = RegressionEvaluator(labelCol="Purchase",metricName="rmse") 
evaluator.evaluate(grid_model.transform(train_data_encoded))

2974.408913947976

---

Get the best model parameters.

---

In [88]:
# extract the best model parameters dictionary
param_dict = grid_model.bestModel.extractParamMap()

In [89]:
param_dict

{Param(parent='RandomForestRegressor_5536294304f1', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='RandomForestRegressor_5536294304f1', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='RandomForestRegressor_5536294304f1', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestRegressor_5536294304f1', name='featuresCol', doc='features column name'): 'feature_vector',
 Param(parent='RandomForestRegressor_5536294304f1',

In [90]:
# created a filtered dictionary
final_dict = {}
for k, v in param_dict.items():
    final_dict[k.name] = v

In [91]:
# get the best elastic net parameter
final_dict["maxDepth"]

15

In [92]:
# get the best regularization parameter
final_dict["numTrees"]

40


----
----


### `Machine Learning Pipelines in Spark`

----


 *   `Transformer` 
 > - It transforms the input data (X) in some ways.
 > - Implements a method `transform()`, which converts one DataFrame into another, generally by appending one or more columns.
 > - It includes `feature transformers` and `learned models`.
 > > - `Feature transformer` should take a DataFrame, read a column (e.g., text), map it into a new column (e.g., feature vectors), and output a new DataFrame with the mapped column appended.
 > > -  `Learning model` should take a DataFrame, read the column containing feature vectors, predict the label for each feature vector, and output a new DataFrame with predicted labels appended as a column.
 *   `Estimator` 
 > - It predicts a new value (or values) (y) by using the input data (X).
 > - It implements a method `fit()`, which accepts a DataFrame and produces a `Model`, which is a `Transformer`.
 > > - For example, a learning algorithm such as `LogisticRegression` is an `Estimator`, and calling `fit()` trains a `LogisticRegressionModel`, which is a `Model` and hence a `Transformer`.
 * `Pipeline`
 > - It represents a sequence of steps to apply in an ML workflow. Example:
 > > - Stage 1 : Split text into words.
 > > - Stage 2 : Convert words into numeric data.
 > > - Stage 3 : Apply machine learning model on the numeric data.
 > - These steps are represented as `Transformers` or as `Estimators`.
 > - A `Pipeline` is comprised of `Stages`.
 > > - These stages are run in order.
 > > - The input DataFrame is transformed as it passes through each stage.
 > > - Each stage is either a `Transformer` or an `Estimator`.
 
 ---
 
**`Import the Required Libraries`**
 
 ---

In [93]:
## Import the Required Libraries 
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.ml import regression

In [94]:
# reading the train data
train_data = spark.read.csv("ml_assignment/train.csv",inferSchema=True, header=True)

# reading the test data
test_data  = spark.read.csv("ml_assignment/test.csv", inferSchema=True, header=True)

In [95]:
train_data=train_data.dropDuplicates()
test_data=test_data.dropDuplicates()

In [96]:
# from pyspark.ml.feature import StringIndexer
# plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'product_ID')
# labeller = plan_indexer.fit(train_data)

# train_data = labeller.transform(train_data)
# test_data = labeller.transform(test_data)

In [97]:
# train_data.select('Product_ID').show()

---
### `Define the stages of the Pipeline`

  * **STAGE 1**: [Transformer] Fill null values with in each column
  * **STAGE 2**: [Estimator] Label Encode: Age
  * **STAGE 4**: [Estimator] Label Encode: Gender
  * **STAGE 5**: [Estimator] Label Encode: City_Category
  * **STAGE 6**: [Estimator] Label Encode: Stay_In_Current_City_Years
  * **STAGE 7**: [Estimator] OHE: Age,Gender,City_Category,Stay_In_Current_City_Years
  * **STAGE 8**: [Transformer] Create Feature: Total Click per Publisher ID, Total Click per Campaign ID
  * **STAGE 9**: Transformer] Create Vector [Marital_Status, gender_ohe, age_ohe, city_category_ohe, product_category_1, product_category_2, product_category_3,]
  * **STAGE 10**: [Estimator] Predict sales Using the Linear Regression





----
#### `Custom Transformer to fill Null Values`

* Fill null value with mode from respective column.
---

In [98]:
# custom transformer to fill null values

class nullValuesTransformer(Transformer):
    
    def __init__(self, dataframe = None):
        self.dataframe = dataframe
    
    def _transform(self, dataframe):
        dataframe = dataframe.fillna({
            "Product_Category_2" : 8,
            "Product_Category_3": 16,
        })
        
        return dataframe

---


#### **`Define the Stages`**

---

In [99]:
# Stage 1 - replace null values
stage_1 = nullValuesTransformer()

# Stage 2 - label encode Age column
stage_2 = StringIndexer(inputCol= "Age", outputCol= "age_le") 

# Stage 3 - label encode Gender column
stage_3 = StringIndexer(inputCol= "Gender", outputCol= "gender_le")

# Stage 4 - label encode City_Category column
stage_4 = StringIndexer(inputCol= "City_Category", outputCol= "city_category_le")

# Stage 5 - label encode Stay_In_Current_City_Years column
stage_5 = StringIndexer(inputCol= "Stay_In_Current_City_Years", outputCol= "stay_in_current_city_years_le")

 # Stage 6 - One Hot Encode columns
stage_6 = OneHotEncoderEstimator(inputCols= ["age_le",  "gender_le", "city_category_le", "stay_in_current_city_years_le"], 
                        outputCols= ["age_ohe",  "gender_ohe", "city_category_ohe", "stay_in_current_city_years_ohe"])

# Stage 7 - Create vector from the columns
stage_7 = VectorAssembler(inputCols= ["Marital_Status",
                                      "gender_ohe",
                                      "age_ohe",
                                      "city_category_ohe",
                                      "stay_in_current_city_years_ohe",
                                      "Product_Category_1",
                                      'Product_Category_2',
                                      'Product_Category_3'],

                         outputCol=  "feature_vector")

# Stage 8 - Train ML model
stage_8 = regression.LinearRegression(featuresCol= "feature_vector", labelCol= "Purchase")

In [101]:
from pyspark.ml.regression import RandomForestRegressor
# from xgboost import XGBRegressor

---

#### **`Define the Pipeline`**

----

In [102]:
# Define pipeline
pipeline = Pipeline(stages= [stage_1,
                             stage_2,
                             stage_3,
                             stage_4,
                             stage_5,
                             stage_6,
                             stage_7,
                             stage_8])

---

### `FIT AND TRANSFORM THE PIPELINE`

---

In [103]:
# fit the pipeline with the training data
pipeline_model = pipeline.fit(train_data)

# transform data
final_data = pipeline_model.transform(train_data)

In [104]:
## The prediction columns give the final prediction of the each record.
final_data.select("User_ID", "Purchase", "prediction").show(10)

+-------+--------+------------------+
|User_ID|Purchase|        prediction|
+-------+--------+------------------+
|1005733|   20173|  8894.16869832875|
|1005763|    5948|  7332.05200616034|
|1005776|   16145| 8611.449903199169|
|1005892|    6447|11461.125295105072|
|1005896|    6903|  9142.32888823947|
|1005916|    4524| 7243.432309924363|
|1005954|   13371|11707.597847815692|
|1006001|   16626|4079.0333730525726|
|1006020|   18986| 7462.647981394239|
|1006034|   14290|7092.2106571772665|
+-------+--------+------------------+
only showing top 10 rows



---

#### `TRANSFORM THE TEST DATA`

---

In [105]:
final_test_data = pipeline_model.transform(test_data)

In [106]:
final_test_data.select("User_ID", "prediction").show(10)

+-------+------------------+
|User_ID|        prediction|
+-------+------------------+
|1004333|10171.970185642247|
|1004344| 9928.341702617017|
|1005108| 8705.495393537345|
|1005126| 9306.711209926105|
|1005184|12357.249251508574|
|1005618|11065.235757683005|
|1005831| 9712.568353911254|
|1006016| 9309.478953665905|
|1000194| 8704.077685216555|
|1000401|  7866.32436155466|
+-------+------------------+
only showing top 10 rows



### RandomForestRegressor

In [107]:
stage_8 = regression.RandomForestRegressor(featuresCol= "feature_vector",  labelCol="Purchase")
# RandomForestRegressor
# stage_8 = regression.LinearRegression(featuresCol= "feature_vector", labelCol= "Purchase")

In [108]:
pipeline = Pipeline(stages= [stage_1,
                             stage_2,
                             stage_3,
                             stage_4,
                             stage_5,
                             stage_6,
                             stage_7,
                             stage_8])

In [109]:
# fit the pipeline with the training data
pipeline_model = pipeline.fit(train_data)

# transform data
final_data = pipeline_model.transform(train_data)

In [110]:
## The prediction columns give the final prediction of the each record.
final_data.select("User_ID", "Purchase", "prediction").show(10)

+-------+--------+------------------+
|User_ID|Purchase|        prediction|
+-------+--------+------------------+
|1003934|    3233| 7676.642549397426|
|1003951|   11423|13851.081414726155|
|1004085|    6179| 7392.062624473605|
|1004085|    9987| 7392.062624473605|
|1004120|    3643| 6830.163459904829|
|1004144|   10079|10907.889218440354|
|1004161|    1979| 6774.953314816991|
|1004183|    8701| 6774.953314816991|
|1004220|    7080|  6776.83638229648|
|1004276|   10846| 9441.981941666798|
+-------+--------+------------------+
only showing top 10 rows



In [111]:
final_test_data = pipeline_model.transform(test_data)

In [112]:
final_test_data.select("User_ID", "prediction").show(10)

+-------+------------------+
|User_ID|        prediction|
+-------+------------------+
|1001536|14474.086618869982|
|1001737| 7871.654885363244|
|1001831|11591.816684048372|
|1003191| 7865.750632144918|
|1003216| 7904.339363412752|
|1003227| 6869.981750300331|
|1003230|13299.871364640072|
|1003280| 7676.642549397426|
|1003589| 7095.651645822142|
|1003929|13007.018372148981|
+-------+------------------+
only showing top 10 rows

